# Deep Convolutional Neural Network for Makam Recognition
10-fold cross validation

### Library importing for file reading and preprocessing

In [1]:
import glob
import os
import json
import numpy as np
from mutagen.mp3 import MP3
import librosa
from scipy.interpolate import interp2d 
from sklearn.preprocessing import normalize
import warnings
from math import floor
import dill

### Pickle session

In [ ]:
dill.dump_session('cnn10fold.db')

In [ ]:
dill.load_session('cnn10fold.db')

### File reading using provided folds and preprocessing

In [2]:
#--------PARAMETERS--------#

#Choose truncation option
# 1 : 15 seconds starting from the start
# 2 : 15 seconds around the middle
# 3 : 15 seconds from the end
trunc_option = 2

#Choose bins and bins per octave
n_bins = 371
bins_per_octave = 53

#--------------------------#

#Ignore mp3 read warning
warnings.filterwarnings("ignore", message="PySoundFile failed. Trying audioread instead.")

#Arrays containing all constant-Q transforms of the soundfiles per fold
X_train = [[],[],[],[],[],[],[],[],[],[]]
X_test = [[],[],[],[],[],[],[],[],[],[]]

#Array containing all makam labels per fold
y_train = [[],[],[],[],[],[],[],[],[],[]]
y_test = [[],[],[],[],[],[],[],[],[],[]]

#Import folds
f = open("./dlfm_makam_recognition_data/folds.json")
folds = json.load(f)

#Compute cqts and import labels
for i in range(len(folds)): #in number of folds
    
    
    #compute all training cqts and import train labels
    for source in range(len(folds[i][1]["training"]["sources"])):
        
        file_name = ("./soundfiles/" + folds[i][1]["training"]["sources"][source] + ".mp3")

        #get file duration for loading
        try:
            audio = MP3(file_name)
        except:
            print("Problem with:", file_name)
            continue
        audio = MP3(file_name)
        duration = audio.info.length
        
        #load 15 seconds from the mp3 according to truncation option
        if (trunc_option == 1): #15 seconds form the start
            #load soundfile
            y, sr = librosa.core.load(file_name, duration=15)
            #compute the constant-Q transform from audio signal
            cqt = librosa.core.cqt(y, sr, n_bins=n_bins, bins_per_octave=bins_per_octave) #up to C7 in 53TET
            
        if (trunc_option == 2): #15 seconds around the center
            offset = (duration-15)/2
            #load soundfile
            y, sr = librosa.core.load(file_name, offset=offset, duration=15)
            #compute the constant-Q transform from audio signal
            cqt = librosa.core.cqt(y, sr, n_bins=n_bins, bins_per_octave=bins_per_octave) #up to C7 in 53TET
            
        if (trunc_option == 3): #15 seconds from the end
            offset = duration-15
            #load soundfile
            y, sr = librosa.core.load(file_name, offset=offset, duration=15)
            #compute the constant-Q transform from audio signal
            cqt = librosa.core.cqt(y, sr, n_bins=n_bins, bins_per_octave=bins_per_octave) #up to C7 in 53TET
        
        X_train[i].append(cqt)
        
        #import makam training label
        y_train[i].append(folds[i][1]["training"]["modes"][source])
        
        
    #compute all testing cqts and import test labels
    for ref in range(len(folds[i][1]["testing"])):
        
        file_name = ("./soundfiles/" + folds[i][1]["testing"][ref]["source"] + ".mp3")

        #get file duration for loading
        try:
            audio = MP3(file_name)
        except:
            print("Problem with:", file_name)
            continue
        duration = audio.info.length
        
        #load 15 seconds from the mp3 according to truncation option
        if (trunc_option == 1): #15 seconds form the start
            #load soundfile
            y, sr = librosa.core.load(file_name, duration=15)
            #compute the constant-Q transform from audio signal
            cqt = librosa.core.cqt(y, sr, n_bins=n_bins, bins_per_octave=bins_per_octave) #up to C7 in 53TET
            
        if (trunc_option == 2): #15 seconds around the center
            offset = (duration-15)/2
            #load soundfile
            y, sr = librosa.core.load(file_name, offset=offset, duration=15)
            #compute the constant-Q transform from audio signal
            cqt = librosa.core.cqt(y, sr, n_bins=n_bins, bins_per_octave=bins_per_octave) #up to C7 in 53TET
            
        if (trunc_option == 3): #15 seconds from the end
            offset = duration-15
            #load soundfile
            y, sr = librosa.core.load(file_name, offset=offset, duration=15)
            #compute the constant-Q transform from audio signal
            cqt = librosa.core.cqt(y, sr, n_bins=n_bins, bins_per_octave=bins_per_octave) #up to C7 in 53TET
        
        X_test[i].append(cqt)
        
        #import makam testing label
        y_test[i].append(folds[i][1]["testing"][ref]["mode"])
        
    print("Fold", i, "loaded.")

X_train = np.asarray(X_train)
X_test = np.asarray(X_test)
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

Problem with: ./soundfiles/1e716ab1-3501-43c8-807b-4fc6180a91f1.mp3
Problem with: ./soundfiles/5a44e069-3b0e-4a28-b515-b7b257b6373a.mp3
Problem with: ./soundfiles/653f8154-b374-4713-99ee-f649f9582b99.mp3
Problem with: ./soundfiles/75c034d1-11e0-4851-93d8-a07ba41fae81.mp3
./soundfiles/b351600d-1ed2-4c69-9662-1677d4cb0fa7.mp3
./soundfiles/bca7c2e4-3263-4022-9de2-42fc70848c85.mp3
./soundfiles/71f8bdfc-6181-43b0-a993-29badae80ddd.mp3
./soundfiles/035d177c-72dd-49c5-b0ca-94c808bfb4c0.mp3
./soundfiles/3c0f63ff-95a3-45eb-9be8-6ef811760ab3.mp3
./soundfiles/f4a426fb-cf87-4fa4-9ee7-a230f4e7307c.mp3
./soundfiles/dce94bb4-447a-439c-a1d6-20e8551aa85f.mp3
./soundfiles/cfafca28-1bf7-431b-8440-a61c1bdf8e58.mp3
./soundfiles/d69470ef-dee5-4a11-9098-9135be5288d7.mp3
./soundfiles/d726cd51-fda8-467e-aa08-3d75f2fffc07.mp3
./soundfiles/e9d8706e-913d-4a64-bd9d-7324ca4ddd41.mp3
./soundfiles/4bd265f0-06a5-46d8-b727-c2e799b1cdf8.mp3
./soundfiles/feda89e3-a50d-4ff8-87d4-c1e531cc1233.mp3
./soundfiles/70a9b17d-0ac8

KeyboardInterrupt: 

### Library importing for deep learning

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, BatchNormalization, Dropout
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

### Data preparation

In [ ]:
#one-hot encode target vector
Y_train = to_categorical(y_train)
Y_test = to_categorical(y_test)
print("Y_train shape:", Y_train.shape)
print("Y_test shape:", Y_test.shape)

### Constructing model

In [ ]:
models = []
for i in range(len(folds)):
    #Network topology
    model = Sequential()

    #3 convolutional layers
    model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', input_shape=(X_train[i][0].shape[0], X_train[i][0].shape[1], 1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())

    model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', input_shape=(X_train[i][0].shape[0], X_train[i][0].shape[1], 1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())

    model.add(Conv2D(filters=64, kernel_size=(2,2), activation='relu', input_shape=(X_train[i][0].shape[0], X_train[i][0].shape[1], 1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())

    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.3))

    #Output layer
    model.add(Dense(20, activation='softmax'))

    #Compile model
    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    model.summary()
    
    models.append(model)

### Model training

In [ ]:
histories = []
for i in range(len(folds)):
    history = model[i].fit(X_train[i], Y_train[i], validation_split=0.25, epochs=10, batch_size=30)
    histories.append(history)

### Model evaluation

In [ ]:
scores = []
y_pred = []
for i in range(len(folds)):
    score = model[i].evaluate(X_test[i], Y_test[i], verbose=1)
    scores.append(score)